## Get Single Event Data from BL4A (multi-run mode)

Version 1.3

T. Charlton

M.R. Fitzsimmons

06.22.2020

Added report of mean proton charge per spin state.

Added dictionary functionality.

06.28.2020 Add process variable for MAGH

## This Notebook facilitates extraction of single event mode data from multiple runs
This notebook needs to be run on analysis.sns.gov.  The files once written can be transferred to a local machine for further processing.

    1) Sequentially loads many nexus event files.
    2) Obtains per pulse proton charge for each event.
    3) Removes proton flash.
    4) Filter events based on polarization state.
    5) Writes information in text and python binary files.

## User provides list of run numbers to process. 
    1) User provides directory path to IPTS experiment.
    2) User provides a list of run numbers in the array called "run_numbers".

In [27]:
import os
%cd ~
cwd_path = os.getcwd()
print('Current working directory: %s'%cwd_path)

#Example:
#directory_path = cwd_path+r'/data/SNS/REF_M/IPTS-23100/' # user input
#run_numbers = ['32400', '32398'] # user input

#IPTS-23100
#directory_path = cwd_path+r'/data/SNS/REF_M/IPTS-23100/' # user input
#run_numbers = ['32400', '32398','32411','32412','32413','32414','32415','32416' ] # user input
#run_numbers = ['32397'] # user input

#IPTS-24314 Liz
#directory_path = cwd_path+r'/data/SNS/REF_M/IPTS-24314/' # user input
#run_numbers = ['34601','34605','34602','34603','34636','34637',
#               '34638','34639','34668','34669','34670','34671','34672',
#               '34644', '34645','34673' ] # user input

#IPTS-24772 Misha
directory_path = cwd_path+r'/data/SNS/REF_M/IPTS-24772/' # user input
#run_numbers = ['35529','35530','35531','35532','35533','35534',
#               '35535','35536','35537'] # user input
#run_numbers = ['35538','35539','35540','35542','35543','35544','35545'] # user input
run_numbers = ['35547','35548','35549','35550','35551','35552','35553'] # user input
run_numbers = ['35558'] # user input
#run_numbers = ['35554','35555','35556','35557','35558','35559'] # user input


#IPTS-18520
# Fails for this IPTS because the experiment was taken pre-epics transformation.

#IPTS-21391
#directory_path = cwd_path+r'/data/SNS/REF_M/IPTS-21391/' # user input
#run_numbers = ['35096', '35097','35098' ] # user input
#run_numbers = ['35000'] # user input 100G 


/SNS/users/mf3
Current working directory: /SNS/users/mf3


## Code to import required libraries (run in a Mantid compatible kernal)

In [28]:
import sys, math, getopt, glob, operator
import numpy as np
import numpy.ma as ma 
np.warnings.filterwarnings('ignore')
import mantid
from mantid.simpleapi import *
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mantid import plots
from datetime import datetime

# Function to identify states of the two flippers for an event.
def filter_cross_sections(ws):
    """
        Filter events according to an aggregated state log.
        :param str file_path: file to read

        BL4A:SF:ICP:getDI

        015 (0000 1111): SF1=OFF, SF2=OFF, SF1Veto=OFF, SF2Veto=OFF
        047 (0010 1111): SF1=ON, SF2=OFF, SF1Veto=OFF, SF2Veto=OFF
        031 (0001 1111): SF1=OFF, SF2=ON, SF1Veto=OFF, SF2Veto=OFF
        063 (0011 1111): SF1=ON, SF2=ON, SF1Veto=OFF, SF2Veto=OFF
        
        From Mathieu Doucet.
    """
    state_log = "BL4A:SF:ICP:getDI"
    states = {'Off_Off': 15,
              'On_Off': 47,
              'Off_On': 31,
              'On_On': 63}
    cross_sections = []
    cross_sections_dict = {}

    for pol_state in states:
        try:
            _ws = FilterByLogValue(InputWorkspace=ws, 
                                   LogName=state_log, 
                                   TimeTolerance=0.1,
                                   MinimumValue=states[pol_state],
                                   MaximumValue=states[pol_state],
                                   LogBoundary='Left',
                                   OutputWorkspace='%s_%s'%(ws.getRunNumber(),
                                                            pol_state))
            _ws.getRun()['cross_section_id'] = pol_state
            if _ws.getNumberEvents() > 0:
                cross_sections.append(_ws)
                cross_sections_dict[pol_state] =_ws
        except:
            mantid.logger.error("Could not filter %s: %s"%(pol_state,
                                                           sys.exc_info()[1]))

    return cross_sections, cross_sections_dict


## Main loop 
    1) Get data for the run.
    2) Remove flash.
    3) Separate into states.
    4) Write files.
    5) Repeat

In [29]:
# All output files will be designated by run number, followed by:
output_file_base = '_events'

_nruns = len(run_numbers)

for _runid in range(_nruns):
    InputFile = directory_path+r'nexus/REF_M_'+run_numbers[_runid]+'.nxs.h5'
    print('Processing: %s'%InputFile)
    OutputFile = directory_path+r'shared/'+run_numbers[_runid]+output_file_base
    
# Get the data
    RawEvents=LoadEventNexus(InputFile)
    RawEvents=Rebin(RawEvents, "2e4, 1., 6e4", PreserveEvents=True) # so that we can plot things in more that one time bin
    
# Remove the proton flash    
    NoProtonFlash=RemovePromptPulse(RawEvents, Width=300,Frequency=60.0)   
    
# Get the proton charge per pulse and absolute time for each pulse.
    pc_values = NoProtonFlash.getRun()['proton_charge'].value
    pc_times  = NoProtonFlash.getRun()['proton_charge'].times    
    
# Filter flipper states
    (ListOfFilteredCrossSections,
     DictOfFilteredCrossSections)=filter_cross_sections(NoProtonFlash)
    # Note: Since the list and the dictionary point to the same set of 
    #       Mantid workspaces, changing one will change the other.
    
# Calculate mean proton charge over entire measurement in order to filter out outliers
   
    if True:   # Choose your way to calculate the mean
        #True.  = > Use numpy
        # Exclude pc values corresponding to zero which occur often if not running at 60Hz, or not running
        _index = np.argwhere(pc_values>0)
        mean_pc = np.mean(pc_values[_index])
        rms_width = np.std(pc_values[_index])
        
    else: # False => Use method below
        charge_sum = np.double(0.)
        charge2_sum = np.double(0.)
        n_sum = np.int32(0)
        for K in DictOfFilteredCrossSections.keys(): # As long as we don't  
                                                     # care about what order   
                                                     # we take the cross sections
                                                     # in we can do this.
            for x in range(256*304):
                f=DictOfFilteredCrossSections[K].getSpectrum(x) # spectrum for a single pixel vs. time
                tofs=f.getTofs(); PulseTimes = f.getPulseTimes(); 
                for i in range(f.getNumberEvents()): 
                    t = mantid.kernel.DateAndTime(PulseTimes[i]).to_datetime64()
                    _index = np.searchsorted(pc_times, t)
                    charge = pc_values[_index]
                    charge_sum = charge_sum + charge
                    charge2_sum = charge2_sum + charge*charge
                    n_sum = n_sum + 1
        mean_pc = charge_sum/n_sum # mean per pulse
        rms_width = np.sqrt((charge2_sum-2*mean_pc*charge_sum+n_sum*mean_pc*mean_pc)/(n_sum-1))
    print('Mean proton charge: %.2f (%.2f)'%(mean_pc,rms_width))

# Write event lists while excluding outliers based on proton current
    n_sig = 4 # accept events for frames having proton charge within 4-sigma
    for K in DictOfFilteredCrossSections.keys(): 
        OFile = open(OutputFile+'_%s.txt'%(K), 'w')
        print('Writing: '+OutputFile+'_%s.txt'%(K))
        OutList =[] ; OutStrings=[]
        for x in range(256*304):
            f=DictOfFilteredCrossSections[K].getSpectrum(x) # spectrum for a single pixel vs. time
            tofs=f.getTofs(); PulseTimes = f.getPulseTimes(); 
            for i in range(f.getNumberEvents()): 
                t = mantid.kernel.DateAndTime(PulseTimes[i]).to_datetime64()
                _index = np.searchsorted(pc_times, t)
                charge = pc_values[_index]
                dt64 = np.datetime64(pc_times[_index])
# OutList contains n-events defined by absolute time of t0 in ns and string forms, tof (micro-s), pixel-id, proton charge
# Here apply filter, only append if within 1-sigma of mean proton charge
                if np.abs(charge-mean_pc) < n_sig/2*rms_width:        
                    OutList.append((dt64.astype(datetime),tofs[i], x, charge))
                    OutStrings.append("%s\t%.10f\t%d\t%.10f\n"%(PulseTimes[i],
                                   tofs[i], x, charge))    
        OutStrings.sort()
        for item in OutStrings: 
            OFile.write(item)
        OFile.close()
        OutList.sort()    
        t0_time = np.zeros(len(OutList), dtype=np.uint64)
        t0_time_string = []
        tof = np.zeros(len(OutList), dtype=np.float32)
        pixel_id = np.zeros(len(OutList), dtype=np.uint32)
        pcharge = np.zeros(len(OutList), dtype=np.float32)
        for i in range(len(OutList)):
            x = OutList[i]
            t0_time[i] = x[0]
            t0_time_string.append(np.datetime64(x[0],'ns'))
            tof[i] = x[1]
            pixel_id[i] = x[2]
            pcharge[i] = x[3]    
        np.savez(OutputFile+'_%s.npz'%(K),t0_time,t0_time_string,tof,pixel_id,pcharge)

# Write slow log data
    RunInfo=RawEvents.getRun()
    instrument = RawEvents.getInstrument()
    source = instrument.getSource()
    sample = instrument.getSample()
    Parameters = instrument.getParameterNames()
    Source_Sample_Distance = sample.getDistance(source) # m
    Det=instrument.getDetector(int(256*304/2))  # distance from sample to center of detector
    Sample_Detector_Center_Distance = Det.getDistance(sample) # m
    pixel_to_coordinate = np.zeros((256*304,3), dtype=np.float32)
    for i in range(256*304):
        Det=instrument.getDetector(i)  # i is the detetector element index
        PixelPos = Det.getPos()
        pixel_to_coordinate[i,0]= Det.getDistance(sample) # m
        pixel_to_coordinate[i,1]= np.arctan(float(PixelPos[0])/float(PixelPos[2])) # radians
        pixel_to_coordinate[i,2]= np.arctan(float(PixelPos[1])/float(PixelPos[2])) # radians
    OutStrings2 = []
    OutStrings2.append('Source to sample distance: %s [m]\n'%Source_Sample_Distance)
    OutStrings2.append('Sample to PSD center distance: %s [m]\n'%Sample_Detector_Center_Distance)

    TemperatureLog= RunInfo.getLogData('BL4A:SE:Lakeshore:KRDG1')
    OutStrings2.append('Mean temperature: %.2f(%.2f) [K]\n'%(np.mean(TemperatureLog.value[:]),np.std(TemperatureLog.value[:])))

    DANGLELog=RunInfo.getLogData('DANGLE')
    OutStrings2.append('Detector angle: %.2f(%.2f) [degrees]\n'%(np.mean(DANGLELog.value[:]),np.std(DANGLELog.value[:])))

    DANGLE0Log=RunInfo.getLogData('DANGLE0')
    OutStrings2.append('Detector offset angle: %.2f(%.2f) [degrees]\n'%(np.mean(DANGLE0Log.value[:]),np.std(DANGLE0Log.value[:])))
    
    SANGLELog=RunInfo.getLogData('SANGLE')
    OutStrings2.append('Sample angle: %.2f(%.2f) [degrees]\n'%(np.mean(SANGLELog.value[:]),np.std(SANGLELog.value[:])))

    C0Log= RunInfo.getLogData('BL4A:CS:ExpPlan:DirectPixel')
    OutStrings2.append('Direct Beam Pixel: %s\n'%np.mean(C0Log.value[:]))

    ChopperFreqLog= RunInfo.getLogData('BL4A:Chop:Gbl:Speed:Req')
    OutStrings2.append('Chopper Frequency: %s [Hz]\n'%np.mean(ChopperFreqLog.value[:]))

    LambdaBarLog= RunInfo.getLogData('BL4A:Chop:Gbl:Wavelength:Req')
    OutStrings2.append('Mean wavelength: %s [Angstroms]\n'%np.mean(LambdaBarLog.value[:]))

    EMagLog= RunInfo.getLogData('BL4A:SE:Bruker:FieldSP')
    OutStrings2.append('Electromagnet induction: %.4f (%.4f) [T]\n'%(np.mean(EMagLog.value[:]),np.std(EMagLog.value[:])))

    try:
        MagHSPLog= RunInfo.getLogData('BL4A:SE:MagH:SP')
        OutStrings2.append('MagH induction (set point): %.4f (%.4f) [T]\n'%(np.mean(MagHSPLog.value[:]),np.std(MagHSPLog.value[:])))
        MagHLog= RunInfo.getLogData('BL4A:SE:MagH:ReadField')
        OutStrings2.append('MagH induction (actual): %.4f (%.4f) [T]\n'%(np.mean(MagHLog.value[:]),np.std(MagHLog.value[:])))
        MagH = True
    
    except RuntimeError:
        print('No MagH process variable in nexus file.')
        MagH = False # new S.E. not all files have this variable

    OFile = open(OutputFile+'_log.txt', 'w')
    print('Writing: '+OutputFile+'_log.txt')
    for item in OutStrings2: 
        OFile.write(item)
    OFile.close()

    if MagH:
        np.savez(OutputFile+'_log.npz',pixel_to_coordinate,
                 Source_Sample_Distance,
                 Sample_Detector_Center_Distance,
                 TemperatureLog.times,TemperatureLog.value,
                 DANGLELog.times,DANGLELog.value,
                 DANGLE0Log.times,DANGLE0Log.value,
                 SANGLELog.times,SANGLELog.value,
                 C0Log.times,C0Log.value,
                 ChopperFreqLog.times,ChopperFreqLog.value,
                 LambdaBarLog.times,LambdaBarLog.value,
                 EMagLog.times,EMagLog.value,
                 MagHSPLog.times,MagHSPLog.value,
                 MagHLog.times,MagHLog.value)
    else:
        np.savez(OutputFile+'_log.npz',pixel_to_coordinate,
                 Source_Sample_Distance,
                 Sample_Detector_Center_Distance,
                 TemperatureLog.times,TemperatureLog.value,
                 DANGLELog.times,DANGLELog.value,
                 DANGLE0Log.times,DANGLE0Log.value,
                 SANGLELog.times,SANGLELog.value,
                 C0Log.times,C0Log.value,
                 ChopperFreqLog.times,ChopperFreqLog.value,
                 LambdaBarLog.times,LambdaBarLog.value,
                 EMagLog.times,EMagLog.value)


Processing: /SNS/users/mf3/data/SNS/REF_M/IPTS-24772/nexus/REF_M_35558.nxs.h5
Mean proton charge: 23196382.09 (86088.99)
Writing: /SNS/users/mf3/data/SNS/REF_M/IPTS-24772/shared/35558_events_Off_Off.txt
Writing: /SNS/users/mf3/data/SNS/REF_M/IPTS-24772/shared/35558_events_log.txt
